In [1]:
#!pip install psycopg2

In [1]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2 as ps

In [2]:
conn = ps.connect(
    dbname = 'postgres',
    user = 'postgres',
    password = 'Castro@1990',
    host = 'localhost',
    port = '5432'
)

In [3]:
sql = """
SELECT * FROM dbo.pnad2020
"""

In [4]:
df_01 = sqlio.read_sql_query(sql, conn)

C:\Users\Gabriel\AppData\Local\Temp\ipykernel_24480\1657119978.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_01 = sqlio.read_sql_query(sql, conn)


In [5]:
df_01.head()

,ano,uf,capital,rm_ride,v1008,v1012,v1013,v1016,estrato,upa,...,f002a1,f002a2,f002a3,f002a4,f002a5,f0061,f006,a006a,a006b,a007a
0,2020,11,11,None,1,4,9,5,1110011,110015970,...,1,1,1,2,1,1,1,None,None,None
1,2020,11,11,None,1,4,9,5,1110011,110015970,...,1,1,1,2,1,1,1,None,None,None
2,2020,11,11,None,1,4,9,5,1110011,110015970,...,1,1,1,2,1,1,1,None,None,None
3,2020,11,11,None,1,4,9,5,1110011,110015970,...,1,1,1,2,1,1,1,None,None,None
4,2020,11,11,None,2,1,9,5,1110011,110015970,...,1,1,1,2,1,1,1,None,None,None


In [6]:
df_01["f0022"].unique()

array([None, '5', '2', '4', '3', '1', '0', '6'], dtype=object)

In [7]:
sql = """
SELECT uf, f0022, COUNT(f0022) AS "faixa_do_aluguel"
FROM dbo.pnad2020
GROUP BY
uf, f0022
ORDER BY
faixa_do_aluguel DESC
"""

In [8]:
df_02 = sqlio.read_sql_query(sql, conn)

C:\Users\Gabriel\AppData\Local\Temp\ipykernel_24480\2735102659.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_02 = sqlio.read_sql_query(sql, conn)


In [9]:
dict_uf = {"11":"Rondônia",
           "12":"Acre",
           "13":"Amazonas",
           "14":"Roraima",
           "15":"Pará",
           "16":"Amapá",
           "17":"Tocantins",
           "21":"Maranhão",
           "22":"Piauí",
           "23":"Ceará",
           "24":"Rio Grande do Norte",
           "25":"Paraíba",
           "26":"Pernambuco",
           "27":"Alagoas",
           "28":"Sergipe",
           "29":"Bahia",
           "31":"Minas Gerais",
           "32":"Espírito Santo",
           "33":"Rio de Janeiro",
           "35":"São Paulo",
           "41":"Paraná",
           "42":"Santa Catarina",
           "43":"Rio Grande do Sul",
           "50":"Mato Grosso do Sul",
           "51":"Mato Grosso",
           "52":"Goiás",
           "53":"Distrito Federal"}

dict_salario = {"0":"1-100",
                "1":"101-300",
                "2":"301-600",
                "3":"601-800",
                "4":"801-1.600",
                "5":"1.601-3.000",
                "6":"3.001-10.000",
                "7":"10.001-50.000",
                "8":"50.001-100.000",
                "9":"Mais de 100.000",
                "88":"Aluguel a vencer",
                None:"Não aplicável"}

In [10]:
df_02_replaced = df_02.replace({"uf":dict_uf, "f0022":dict_salario})

In [11]:
df_02_replaced = df_02_replaced.rename(columns={"uf":"Estado","f0022":"Faixa do Aluguel","faixa_do_aluguel":"Quantidade"})

In [12]:
df_02_replaced.head()

,Estado,Faixa do Aluguel,Quantidade
0,Minas Gerais,301-600,7113
1,São Paulo,301-600,6535
2,São Paulo,801-1.600,5650
3,Rio de Janeiro,301-600,4779
4,Goiás,301-600,4272


In [13]:
df_02_replaced.groupby(by="Faixa do Aluguel")["Quantidade"].sum().sort_values(ascending=False)

Faixa do Aluguel
301-600          66775
801-1.600        31639
601-800          26613
101-300          26364
1.601-3.000       7323
3.001-10.000      1375
1-100             1059
Não aplicável        0
Name: Quantidade, dtype: int64

In [14]:
df_02_replaced.where(df_02_replaced["Faixa do Aluguel"] == "301-600").nlargest(n=5,columns=["Quantidade"])

,Estado,Faixa do Aluguel,Quantidade
0,Minas Gerais,301-600,7113.0
1,São Paulo,301-600,6535.0
3,Rio de Janeiro,301-600,4779.0
4,Goiás,301-600,4272.0
7,Paraná,301-600,3893.0


In [15]:
df_02_replaced.where(df_02_replaced["Faixa do Aluguel"] == "801-1.600").nlargest(n=5,columns=["Quantidade"])

,Estado,Faixa do Aluguel,Quantidade
2,São Paulo,801-1.600,5650.0
5,Rio de Janeiro,801-1.600,4231.0
10,Santa Catarina,801-1.600,3388.0
12,Minas Gerais,801-1.600,2952.0
22,Paraná,801-1.600,2079.0


In [16]:
df_02_replaced.where(df_02_replaced["Faixa do Aluguel"] == "601-800").nlargest(n=5,columns=["Quantidade"])

,Estado,Faixa do Aluguel,Quantidade
6,São Paulo,601-800,4173.0
15,Rio de Janeiro,601-800,2807.0
19,Minas Gerais,601-800,2623.0
20,Santa Catarina,601-800,2453.0
28,Paraná,601-800,1831.0


In [17]:
df_02_replaced.where(df_02_replaced["Faixa do Aluguel"] == "101-300").nlargest(n=5,columns=["Quantidade"])

,Estado,Faixa do Aluguel,Quantidade
13,Ceará,101-300,2947.0
18,Pernambuco,101-300,2665.0
26,Minas Gerais,101-300,1881.0
27,Alagoas,101-300,1838.0
31,Maranhão,101-300,1619.0


In [18]:
faixa_01 = df_02_replaced.where(df_02_replaced["Faixa do Aluguel"] == "301-600").nlargest(n=5,columns=["Quantidade"])
faixa_02 = df_02_replaced.where(df_02_replaced["Faixa do Aluguel"] == "801-1.600").nlargest(n=5,columns=["Quantidade"])
faixa_03 = df_02_replaced.where(df_02_replaced["Faixa do Aluguel"] == "601-800").nlargest(n=5,columns=["Quantidade"])
faixa_04 = df_02_replaced.where(df_02_replaced["Faixa do Aluguel"] == "101-300").nlargest(n=5,columns=["Quantidade"])

In [19]:
faixas = pd.concat([faixa_01,faixa_02,faixa_03,faixa_04])
faixas

,Estado,Faixa do Aluguel,Quantidade
0,Minas Gerais,301-600,7113.0
1,São Paulo,301-600,6535.0
3,Rio de Janeiro,301-600,4779.0
4,Goiás,301-600,4272.0
7,Paraná,301-600,3893.0
2,São Paulo,801-1.600,5650.0
5,Rio de Janeiro,801-1.600,4231.0
10,Santa Catarina,801-1.600,3388.0
12,Minas Gerais,801-1.600,2952.0
22,Paraná,801-1.600,2079.0


In [20]:
color_list = faixas["Estado"].unique()

In [21]:
import matplotlib.pyplot as plt
import warnings 
  
# Settings the warnings to be ignored 
warnings.filterwarnings('ignore') 

In [ ]:
color_dict={color_list[0]:'red', color_list[1]:'blue', color_list[2]:'green', color_list[3]:'aqua', color_list[4]:'orange',
       color_list[5]:'purple', color_list[6]:'chocolate', color_list[7]:'yellow', color_list[8]:'grey', color_list[9]:'navy'}

fig, (ax1, ax2, ax3, ax4)= plt.subplots(1, 4, sharey=True)
ax1.bar(faixa_01["Estado"], faixa_01["Quantidade"], color=[color_dict[key] for key in faixa_01["Estado"]],  edgecolor='black', zorder=3)
ax2.bar(faixa_02["Estado"], faixa_02["Quantidade"], color=[color_dict[key] for key in faixa_02["Estado"]],  edgecolor='black', zorder=3)
ax3.bar(faixa_03["Estado"], faixa_03["Quantidade"], color=[color_dict[key] for key in faixa_03["Estado"]],  edgecolor='black', zorder=3)
ax4.bar(faixa_04["Estado"], faixa_04["Quantidade"], color=[color_dict[key] for key in faixa_04["Estado"]],  edgecolor='black', zorder=3)
fig.suptitle("PNAD COVID19\nNúmero de entrevistados em diferentes faixas de aluguel por estado", size=16)
ax1.set_title("Aluguel de 301 a 600 reais")
ax2.set_title("Aluguel de 801 a 1.600 reais")
ax3.set_title("Aluguel de 601 a 800 reais")
ax4.set_title("Aluguel de 101 a 300 reais")
ax1.set_xticklabels(faixa_01["Estado"], rotation=45, ha='right')
ax2.set_xticklabels(faixa_02["Estado"], rotation=45, ha='right')
ax3.set_xticklabels(faixa_03["Estado"], rotation=45, ha='right')
ax4.set_xticklabels(faixa_04["Estado"], rotation=45, ha='right')
ax1.grid(axis = 'y', color = 'gray', linestyle = '--', linewidth = 0.8, alpha=0.8, zorder=0)
ax2.grid(axis = 'y', color = 'gray', linestyle = '--', linewidth = 0.8, alpha=0.8, zorder=0)
ax3.grid(axis = 'y', color = 'gray', linestyle = '--', linewidth = 0.8, alpha=0.8, zorder=0)
ax4.grid(axis = 'y', color = 'gray', linestyle = '--', linewidth = 0.8, alpha=0.8, zorder=0)
fig.set_figheight(6)
fig.set_figwidth(15)
fig.text(0.5, -0.01, 'Estado', ha='center', fontdict={'size':16})
fig.text(-0.01, 0.5, 'Número de entrevistados', va='center', rotation='vertical', fontdict={'size':16})
plt.legend()
plt.tight_layout()
plt.show()


In [35]:
import json
import plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
fig_1 = make_subplots(rows=2, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}],[{'type':'domain'}, {'type':'domain'}]])
fig_1.add_trace(go.Pie(labels=faixa_01["Estado"], values=faixa_01["Quantidade"], name="301-600"),
            1, 1)
fig_1.add_trace(go.Pie(labels=faixa_02["Estado"], values=faixa_02["Quantidade"], name="801-1.600"),
            1, 2)
fig_1.add_trace(go.Pie(labels=faixa_03["Estado"], values=faixa_03["Quantidade"], name="601-800"),
            2, 1)
fig_1.add_trace(go.Pie(labels=faixa_04["Estado"], values=faixa_04["Quantidade"], name="101-300"),
            2, 2)

# Tamanho do buraco da rosca
fig_1.update_traces(hole=0.7, hoverinfo="label+value+percent", textinfo='value')

fig_1.update_layout(title_text="Número de entrevistados em diferentes faixas de aluguel por estado", title_x=0.5, title=dict(font=dict(size=16)),legend=dict(font=dict(size=14)),
                    annotations=[dict(text='301-600<br>Reais', x=0.160, y=0.82, font_size=18, showarrow=False),
                                 dict(text='801-1.600<br>Reais', x=0.9, y=0.82, font_size=18, showarrow=False),
                                 dict(text='601-800<br>Reais', x=0.160, y=0.12, font_size=18, showarrow=False),
                                 dict(text='101-300<br>Reais', x=0.9, y=0.12, font_size=18, showarrow=False)
                                ]
                    )
#plotly.offline.plot(fig_1, filename = 'br_faixa_aluguel.html', auto_open=False)
fig_1.show()

In [51]:
sql = """
SELECT v1013, uf, b0011, b0012, b0015, COUNT(b0011) AS "febre_tosse_dor_cabeca"
FROM dbo.pnad2020
WHERE b0011 = '1' AND b0012 = '1' AND b0015 = '1'
GROUP BY
v1013, uf, b0011, b0012, b0015
ORDER BY
v1013 DESC, febre_tosse_dor_cabeca DESC
"""

In [52]:
df_03 = sqlio.read_sql_query(sql, conn)

In [53]:
df_03.head()

,v1013,uf,b0011,b0012,b0015,febre_tosse_dor_cabeca
0,9,51,1,1,1,87
1,9,52,1,1,1,85
2,9,43,1,1,1,75
3,9,31,1,1,1,69
4,9,35,1,1,1,63


In [54]:
dict_uf_short = {"11":"RO",
                 "12":"AC",
                 "13":"AM",
                 "14":"RR",
                 "15":"PA",
                 "16":"AP",
                 "17":"TO",
                 "21":"MA",
                 "22":"PI",
                 "23":"CE",
                 "24":"RN",
                 "25":"PB",
                 "26":"PE",
                 "27":"AL",
                 "28":"SE",
                 "29":"BA",
                 "31":"MG",
                 "32":"ES",
                 "33":"RJ",
                 "35":"SP",
                 "41":"PR",
                 "42":"SC",
                 "43":"RS",
                 "50":"MS",
                 "51":"MT",
                 "52":"GO",
                 "53":"DF"
                 }

dict_mes = {"9":"Setembro",
            "10":"Outubro",
            "11":"Novembro"
           }

In [55]:
df_03_replaced = df_03.replace({"uf":dict_uf_short,"v1013":dict_mes})
df_03_replaced = df_03_replaced.drop(columns=["b0011", "b0012", "b0015"])
df_03_replaced = df_03_replaced.rename(columns={"v1013":"mes"})

In [56]:
df_03_replaced.head()

,mes,uf,febre_tosse_dor_cabeca
0,Setembro,MT,87
1,Setembro,GO,85
2,Setembro,RS,75
3,Setembro,MG,69
4,Setembro,SP,63


In [57]:
df_03_replaced.to_csv("C:\\Users\\Gabriel\\Downloads\\Outros\\Pós_Tech\\Curso\\Fase III\\Tech-Challenge\\Assets\\DataFrame\\casos_de_febre.csv")

In [58]:
# Recuperando dados geológicos do Brasil
f = open('Assets/JSON/brazil_geo.json',)
geo_json_br = json.load(f)

In [59]:
casos_de_febre = pd.read_csv('Assets/DataFrame/casos_de_febre.csv')

In [ ]:
fig_2 = px.choropleth_mapbox(casos_de_febre,
                                geojson = geo_json_br,
                                locations='uf',
                                featureidkey = 'id',
                                color = 'febre_tosse_dor_cabeca',
                                animation_frame = 'mes',
                                hover_name = 'uf',
                                hover_data = ['febre_tosse_dor_cabeca'],
                                title = 'Casos de febre, tosse e dor de cabeça no Brasil entre Setembro a Novembro de 2020',
                                color_continuous_scale='Viridis',
                                mapbox_style = 'carto-positron',
                                center = {'lat':-14.654012, 'lon': -52.879523},
                                zoom = 3,
                                opacity = 0.9, )
fig_2.update_layout(coloraxis_colorbar_title_text = 'Nº Casos', title_x=0.5, title=dict(font=dict(size=14)))
fig_2.update_geos(fitbounds = 'locations', visible = False)
plotly.offline.plot(fig_2, filename = 'br_mapa_casos_febre.html', auto_play=False, auto_open=False)
fig_2.show()

In [37]:
sql = """
SELECT uf, v1022, COUNT(case v1022 when '1' then 1 else null end) AS "urbana", COUNT(case v1022 when '2' then 1 else null end) AS "rural", COUNT(v1022) AS "total"
FROM dbo.pnad2020
GROUP BY
uf, v1022
ORDER BY
total DESC
"""

In [38]:
df_04 = sqlio.read_sql_query(sql, conn)

In [39]:
df_04.head()

,uf,v1022,urbana,rural,total
0,33,1,81912,0,81912
1,35,1,80956,0,80956
2,31,1,76102,0,76102
3,42,1,56342,0,56342
4,43,1,49633,0,49633


In [40]:
df_04.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   uf      54 non-null     object
 1   v1022   54 non-null     object
 2   urbana  54 non-null     int64 
 3   rural   54 non-null     int64 
 4   total   54 non-null     int64 
dtypes: int64(3), object(2)
memory usage: 2.2+ KB


In [41]:
df_04.head()

,uf,v1022,urbana,rural,total
0,33,1,81912,0,81912
1,35,1,80956,0,80956
2,31,1,76102,0,76102
3,42,1,56342,0,56342
4,43,1,49633,0,49633


In [42]:
df_04_copy = df_04.copy()

In [43]:
dict_v1022 = {
    "1":"urbana",
    "2":"rural"
}

In [44]:
df_04_copy = df_04_copy.drop(columns=["urbana","rural"])
df_04_copy = df_04_copy.replace({"v1022":dict_v1022})
df_04_copy = df_04_copy.rename(columns={"v1022":"situacao"})
df_04_copy = df_04_copy.replace({"uf":dict_uf})
df_04_copy.head()

,uf,situacao,total
0,Rio de Janeiro,urbana,81912
1,São Paulo,urbana,80956
2,Minas Gerais,urbana,76102
3,Santa Catarina,urbana,56342
4,Rio Grande do Sul,urbana,49633


In [45]:
agg_functions = {'uf': 'first', 'situacao': 'first', 'total': 'sum'}
df_04_total = df_04_copy.groupby(df_04_copy['uf']).aggregate(agg_functions)

In [46]:
df_04_total = df_04_total.reset_index(drop=True)
df_04_total = df_04_total.sort_values('total', ascending=False)
df_04_total = df_04_total.drop(columns=["situacao"])
df_04_total.head()

,uf,total
12,Minas Gerais,103096
25,São Paulo,98021
20,Rio de Janeiro,87405
23,Santa Catarina,70990
19,Rio Grande do Sul,66446


In [47]:
df_04_merge = pd.merge(df_04_copy, df_04_total, on="uf")
df_04_merge.head()

,uf,situacao,total_x,total_y
0,Rio de Janeiro,urbana,81912,87405
1,Rio de Janeiro,rural,5493,87405
2,São Paulo,urbana,80956,98021
3,São Paulo,rural,17065,98021
4,Minas Gerais,urbana,76102,103096


In [48]:
df_04_final = df_04_merge.rename(columns={"total_x":"total_por_situacao","total_y":"total"})
df_04_final = df_04_final.sort_values("total",ascending=False)
df_04_final.head()

,uf,situacao,total_por_situacao,total
4,Minas Gerais,urbana,76102,103096
5,Minas Gerais,rural,26994,103096
2,São Paulo,urbana,80956,98021
3,São Paulo,rural,17065,98021
0,Rio de Janeiro,urbana,81912,87405


In [ ]:
fig_3 = px.bar(df_04_final, x="uf", y="total_por_situacao", color="situacao", title="Ditribuição da população da pesquisa por situação",
             labels=dict(uf="Estado", total_por_situacao="Nº de entrevistados", situacao="Situação"))
fig_3.update_layout(title_x=0.5)
#plotly.offline.plot(fig_3, filename = 'br_distribuicao_populacao.html', auto_play=False, auto_open=False)
fig_3.show()

In [63]:
df_05 = df_03_replaced.drop(columns=["mes"])

In [64]:
df_05

,uf,febre_tosse_dor_cabeca
0,MT,87
1,GO,85
2,RS,75
3,MG,69
4,SP,63
...,...,...
76,TO,18
77,MS,17
78,RN,9
79,SE,7


In [66]:
df_05_mes = df_05.groupby(["uf"])["febre_tosse_dor_cabeca"].sum()
df_05_mes

uf
AC     72
AL     96
AM    136
AP     22
BA     82
CE    116
DF     87
ES    113
GO    193
MA    147
MG    205
MS     81
MT    161
PA    161
PB     71
PE    112
PI     50
PR    150
RJ    197
RN     39
RO     85
RR    100
RS    196
SC    241
SE     40
SP    181
TO     67
Name: febre_tosse_dor_cabeca, dtype: int64

In [68]:
df_05_pop = df_04_final.groupby(["uf"])["total"].first()
df_05_pop

uf
Acre                    18834
Alagoas                 34450
Amapá                    7677
Amazonas                31355
Bahia                   51310
Ceará                   50536
Distrito Federal        19906
Espírito Santo          47511
Goiás                   41977
Maranhão                61200
Mato Grosso             34156
Mato Grosso do Sul      26434
Minas Gerais           103096
Paraná                  59984
Paraíba                 29687
Pará                    35764
Pernambuco              50811
Piauí                   25137
Rio Grande do Norte     23552
Rio Grande do Sul       66446
Rio de Janeiro          87405
Rondônia                16622
Roraima                 16674
Santa Catarina          70990
Sergipe                 24216
São Paulo               98021
Tocantins               15446
Name: total, dtype: int64

In [70]:
dict_estado_uf={"Acre":"AC",
                "Alagoas":"AL",
                "Amapá":"AP",
                "Amazonas":"AM",
                "Bahia":"BA",
                "Ceará":"CE",
                "Distrito Federal":"DF",
                "Espírito Santo":"ES",
                "Goiás":"GO",
                "Maranhão":"MA",
                "Mato Grosso":"MT",
                "Mato Grosso do Sul":"MS",
                "Minas Gerais":"MG",
                "Paraná":"PR",
                "Paraíba":"PB",
                "Pará":"PA",
                "Pernambuco":"PE",
                "Piauí":"PI",
                "Rio Grande do Norte":"RN",
                "Rio Grande do Sul":"RS",
                "Rio de Janeiro":"RJ",
                "Rondônia":"RO",
                "Roraima":"RR",
                "Santa Catarina":"SC",
                "Sergipe":"SE",
                "São Paulo":"SP",
                "Tocantins":"TO"
}

In [72]:
df_05_pop = df_05_pop.to_frame()

In [79]:
df_05_pop = df_05_pop.reset_index()


In [83]:
df_05_pop = df_05_pop.replace({"uf":dict_estado_uf})


In [88]:
df_05_mes = df_05_mes.to_frame()
df_05_mes = df_05_mes.reset_index()

In [113]:
df_05_final = df_05_mes.merge(df_05_pop)

In [114]:
df_05_final["porcentagem"] = df_05_final["febre_tosse_dor_cabeca"]/df_05_final["total"]
df_05_final = df_05_final.sort_values("porcentagem",ascending=False)
df_05_final

,uf,febre_tosse_dor_cabeca,total,porcentagem
21,RR,100,16674,0.005997
20,RO,85,16622,0.005114
12,MT,161,34156,0.004714
8,GO,193,41977,0.004598
13,PA,161,35764,0.004502
6,DF,87,19906,0.004371
26,TO,67,15446,0.004338
2,AM,136,31355,0.004337
0,AC,72,18834,0.003823
23,SC,241,70990,0.003395


In [203]:
import locale
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

In [232]:
fig_04 = go.Figure(
    data=go.Bar(
        x=df_05_final["uf"].head().values,
        y=df_05_final["total"].head().values,
        name="Total entrevistados"
    )
)

fig_04.add_trace(
    go.Scatter(
        x=df_05_final["uf"].head().values,
        y=df_05_final["porcentagem"].head().values,
        yaxis="y2",
        name="Porcentagem casos",
        mode="text+lines+markers",
        texttemplate="%{y}",
        textfont=dict(
                color="black",
                ),
        marker=dict(
            symbol="circle",
            size=50,
        ),
    )
)

fig_04.update_traces(textfont_size=14)


fig_04.update_layout(
    separators=",.",
    title_text="Porcentagem de casos de febre, tosse e dor na cabeça dos cinco estados mais afetados", 
    title_x=0.5,
    title=dict(font=dict(size=14)),
    legend=dict(orientation="h"),
    yaxis=dict(
        title=dict(text="Total de entrevistados"),
        side="left",
    ),
    yaxis2=dict(
        title=dict(text="Porcentagem de casos"),
        side="right",
        overlaying="y",
        tickmode="sync",
        tickformat= ".2%"
    ),
)
plotly.offline.plot(fig_04, filename = 'br_porcentagem_casos.html', auto_play=False, auto_open=False)
fig_04.show()

## Links

###### https://andre-kuniyoshi.medium.com/plotly-choropleth-ainteractive-sao-paulo-cities-map-61089cd7ba2d
###### https://www.kaggle.com/datasets/thiagobodruk/brazil-geojson
###### https://github.com/tbrugz/geodata-br
###### https://plotly.com/python/mapbox-county-choropleth/
###### https://plotly.github.io/plotly.py-docs/generated/plotly.express.choropleth_mapbox.html